Week 2, Day 4 - Use of Tools with openAI - Saurabh Gupta. For any feedback, please mail it to srbmisc@gmail.com, I would love connecting with you to discuss more on this, potentially improving it. Thank you for your read.


# 🔍 LinkedIn Job Search Conversationally with OpenAI and JobSpy

Looking for a job on LinkedIn but tired of manually scrolling through endless listings? This Python project simplifies your job hunt by automating the process of searching LinkedIn job postings based on your **desired role** and **location**.

## 🚀 What It Does

Using the powerful Python scraping library **`jobspy`** [Github Link](https://github.com/speedyapply/JobSpy), this tool fetches **real-time job listings from LinkedIn** tailored to your input. Just provide:


- 🧑‍💻 **Job Title** (e.g., *Data Scientist*, *Frontend Developer*)
- 📍 **Location** (e.g., *New York*, *Remote*, *Berlin*)

…and the script will return up to **3 curated job listings** matching your criteria — complete with **direct links to the job posts** on LinkedIn.

## ✅ Key Features

- 🔎 Real-time LinkedIn job scraping  
- ⚡ Fast and minimalistic – only the top 3 results  
- 🔗 Includes clickable links to the job listings  
- 💬 Simple CLI input — no setup hassles

## 📌 Use Case

Whether you're actively job hunting or just exploring opportunities in your field, this tool helps you **stay updated** with fresh job listings — without opening a browser.


In [ ]:
try:
  from jobspy import scrape_jobs
except:
  %pip install -U python-jobspy
  from jobspy import scrape_jobs

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import pandas as pd

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()


In [ ]:
def callExternalAPI(title, location) -> str:
    jobs = pd.DataFrame(scrape_jobs(
        site_name=["linkedin"],
        search_term=title,
        location=location,
        results_wanted=3,
        hours_old=48
    ))
    print(f"Found {len(jobs)} jobs")
    if len(jobs) > 0:
        df = pd.DataFrame(jobs.loc[:,['title','company','location','date_posted','job_url']])
        df = df.fillna('N/A')
        return df.to_json()
    return 'No results found'

In [ ]:
system_message = ''' You are a helpful assistant for a Jobs Agency who helps users with job listings available.
Give short, courteous answers to the users. Tell the user you can bring max 3 job listings at once. Present the job listings to the user in nice Markdown.
Always be accurate. If you don't know the answer, say so.'''

In [ ]:
getData_function = {
    "name": "callExternalAPI",
    "description": '''Get the Job listings for the provided job title and/or location. Call this whenever the user needs specific job listings data for the title or location that they provide. Following are few examples - 
     1. Give me the jobs available for Software Engineer at Gurugram, India. - In such case, you can call this function by passing the job_title as Software Engineer and job_location as Gurugram, India.
     2. Give me the jobs available for Software Engineer. - In such case, you can call this function by passing the job_title as Software Engineer and job_location as blank string.
     3. Give me the jobs available at Gurugram, India. - In such case, you can call this function by passing the job_title as blank string and job_location as Gurugram, India.
     ''',
    "parameters": {
        "type": "object",
        "properties": {
            "job_title": {
                "type": "string",
                "description": "The title or position or designation of the job the user is interested in",
            },
            "job_location": {
                "type": "string",
                "description": "The location of the job the user is interested in",
            }
        },
        "required": ["job_title","job_location"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": getData_function}]

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    job_title = arguments.get('job_title')
    job_location = arguments.get('job_location')
    jobs = callExternalAPI(job_title,job_location)
    response = {
        "role": "tool",
        "content": json.dumps({"job_title": job_title,"job_location": job_location,"job_listings": jobs}),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()